# 📊 Sales Analysis - Superstore Dataset 

**First Project of the Data Analytics Training Program**
*In partnership with Brainwave Matrix Solutions - 2025*
##### Project Goal: Analyze sales data
###### Tools Used: Python (Pandas, Matplotlib, Seaborn, plotly.express), Jupyter Notebook
###### Data Source: [Superstore Dataset from Kaggle](https://www.kaggle.com/datasets/vivek468/superstore-dataset-final)

# 1. Importing Libraries and Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

In [ ]:
df = pd.read_csv('SampleSuperstore.csv', encoding='latin1')
df.head()

# 2. Data Exploration

In [ ]:
print(df.info())

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
categorical_columns = [cname for cname in df.columns if df[cname].dtype == "object"]
categorical_columns

In [ ]:
print("unique values in Ship Mode: {0}".format(len(df['Ship Mode'].unique().tolist())))
print(df['Ship Mode'].value_counts())

In [ ]:
print("unique values in Segment: {0}".format(len(df['Segment'].unique().tolist())))
print(df['Segment'].value_counts())

In [ ]:
print("unique values in Country: {0}".format(len(df['Country'].unique().tolist())))
print(df['Country'].value_counts())

In [ ]:
print("unique values in State: {0}".format(len(df['State'].unique().tolist())))
print(df['State'].value_counts())

In [ ]:
print("unique values in City: {0}".format(len(df['City'].unique().tolist())))
print(df['City'].value_counts())
df.groupby(['Region', 'State'])['City'].value_counts()

In [ ]:
print("unique values in Region: {0}".format(len(df['Region'].unique().tolist())))
print(df['Region'].value_counts())

In [ ]:
print("unique values in Category: {0}".format(len(df['Category'].unique().tolist())))
print(df['Category'].value_counts())

In [ ]:
print("unique values in Sub Category: {0}".format(len(df['Sub-Category'].unique().tolist())))
print(df['Sub-Category'].value_counts())
df.groupby('Category')['Sub-Category'].value_counts()

In [ ]:
print("unique values in Products: {0}".format(len(df['Product Name'].unique().tolist())))
print(df['Product Name'].value_counts())
df.groupby(['Category', 'Sub-Category'])['Product Name'].value_counts()

# 3. Data Cleaning

In [ ]:
# Fix dates
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date']) 
print(df['Order Date'].dtype)
print(df['Ship Date'].dtype)

In [ ]:
# Add useful columns
df['Unit Price'] = df['Sales'] / df['Quantity']
df['Total Cost'] = df['Sales'] - df['Profit']
df['Total Discount'] = df['Discount'] * df['Quantity']
df['Gross Profit'] = df['Profit'] + df['Total Discount']
df['Profit Margin'] = (df['Profit'] / df['Sales']) * 100

# Processing dates
df['Order Year'] = df['Order Date'].dt.year
df['Order Month'] = df['Order Date'].dt.month
df['Order Day'] = df['Order Date'].dt.day
df['Order Weekday'] = df['Order Date'].dt.day_name()

df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df['Ship Year'] = df['Ship Date'].dt.year
df['Ship Month'] = df['Ship Date'].dt.month
df['Ship Day'] = df['Ship Date'].dt.day
df['Ship Weekday'] = df['Ship Date'].dt.day_name()

df['Shipping Delay'] = (df['Ship Date'] - df['Order Date']).dt.days

In [ ]:
df.head()

# 4. Descriptive analysis (preliminary)

### ---- Net profit calculation ----

In [ ]:
net_profit = df['Profit'].sum()
print(f"Net profit/total loss: ${net_profit:,.2f}")
net_profit

### ---- Winning and losing deals ----

In [ ]:
total_transactions = len(df)
profitable_transactions = len(df[df['Profit'] > 0])
loss_transactions = len(df[df['Profit'] < 0])

print(f"Total Trades: {total_transactions}")
print(f"Profitable Trades: {profitable_transactions} ({profitable_transactions/total_transactions*100:.1f}%)")
print(f"Losing deals: {loss_transactions} ({loss_transactions/total_transactions*100:.1f}%)")

### ---- Profit Distribution by Item  ----

In [ ]:
plt.figure(figsize=(8, 3))
plt.hist(df['Profit'], bins=50, color='lightgreen', edgecolor='black')
plt.axvline(0, color='red', linestyle='dashed', linewidth=1)
plt.title('Profit Distribution by Item')
plt.xlabel('Profit per item')
plt.ylabel('Number of Items')  
plt.show()

### ---- Distribution of major categories  ----

In [ ]:
category_ratio = df['Category'].value_counts(normalize=True)
plt.figure(figsize=(6, 6))
plt.pie(category_ratio.values, labels=category_ratio.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'), shadow=True)
plt.title('Demand ratio by category')
plt.show()

### ---- Top Spending Customers  ----

In [ ]:
customer_analysis = df.groupby(['Customer ID', 'Customer Name']).agg(
    Total_Orders=('Order ID', 'nunique'),
    Total_Spent=('Sales', 'sum')
).reset_index().sort_values('Total_Spent', ascending=False).head(10)

px.treemap(customer_analysis, path=['Customer Name'], values='Total_Spent',
          title='Top 10 Spending Customers')

### ---- Analysis of the most requested states - a relative representation of the top 4 states by number of requests  ----

In [ ]:
States_orders = df['State'].value_counts().head(4)
mylabels = States_orders.values / States_orders.values.sum() *100
plt.pie(States_orders, startangle=90, labels=mylabels.round(1), colors=['#2D3250', '#F6B17A'], shadow=True)
plt.legend(labels=States_orders.index, loc='upper right')
plt.title('Top 4 States by Order Count')
plt.show()

# 🔍*Temporal analysis of data*

## ===== Part One: Order Timing Analysis =====

### ---- 📊Analysis of the most requested time ----

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 10))

# Orders by year
sns.countplot(x='Order Year', data=df, ax=axs[0, 0],hue='Order Year', palette='Blues')
axs[0, 0].set_title('Number of Orders per Year')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Number of Orders')

# Orders by month
sns.countplot(x='Order Month', data=df, ax=axs[0, 1],hue='Order Month', palette='Greens')
axs[0, 1].set_title('Number of Orders per Month')
axs[0, 1].set_xlabel('Month')
axs[0, 1].set_ylabel('Number of Orders')

# Orders by day of the month
sns.countplot(x='Order Day', data=df, ax=axs[1, 0], hue='Order Day', palette='Oranges')
axs[1, 0].set_title('Number of orders per day of the month')
axs[1, 0].set_xlabel('Day')
axs[1, 0].set_ylabel('Number of Orders')

# Orders by days of the week
sns.countplot(x='Order Weekday', data=df, ax=axs[1, 1], order=['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday'],
              hue='Order Weekday', palette='Purples')
axs[1, 1].set_title('Number of requests by day of the week')
axs[1, 1].set_xlabel('Day')
axs[1, 1].set_ylabel('Number of Orders')

plt.tight_layout()
plt.show()

#### 🕵️‍♂️ **Insights from Time-Based Order Analysis:**

1. **Yearly Trend:**
   The number of orders shows a consistent growth over the years, indicating increasing customer engagement and possibly expanding business operations.

2. **Monthly Trend:**
   Certain months, particularly "November" and "December", exhibit a spike in order volume. This could be attributed to seasonal promotions, holidays, or year-end sales.

3. **Daily Trend (Day of Month):**
   Orders appear to be fairly evenly distributed across the days of each month, with slight peaks around the 20th and 21th of the month, which might correlate with salary cycles or specific promotions.

4. **Weekly Trend (Day of Week):**
   Most orders are placed on Monday and Friday, suggesting a pattern in consumer behavior, such as increased shopping at the beginning or end of the week. Lower order volumes on Wednesday could guide targeted marketing efforts.

###  ---- 📊Sales change over time ----

In [ ]:
plt.figure(figsize=(12, 5))
df_grouped_sales = df.groupby('Order Date')['Sales'].sum().reset_index()
sns.lineplot(data=df_grouped_sales, x='Order Date', y='Sales', color='#199fff')
plt.title('Sales change over time')
plt.xlabel('Order Date')
plt.ylabel('Sales')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

Sales show an overall upward trend with noticeable fluctuations, reflecting seasonal demand and promotional campaigns. Monitoring this can guide inventory and sales strategies.

###  ---- 📊Quantity change over time ----

In [ ]:
plt.figure(figsize=(12, 5))
df_grouped_quantity = df.groupby('Order Date')['Quantity'].sum().reset_index()
sns.lineplot(data=df_grouped_quantity, x='Order Date', y='Quantity')
plt.title('Quantity changes over time')
plt.xlabel('Order Date')
plt.ylabel('Quantity')
plt.grid(True, linestyle='--', alpha=0.5)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

The quantity of items ordered follows a pattern similar to sales, with fluctuations indicating changes in demand. Tracking quantity helps identify high-demand periods and optimize stock levels.

###  ---- 📊The average discount changes over time ----

In [ ]:
plt.figure(figsize=(12, 5))
df_grouped_discount = df.groupby('Order Date')['Discount'].mean().reset_index()

sns.lineplot(
    data=df_grouped_discount,
    x='Order Date',
    y='Discount',
    color='#c89ec4', 
    linewidth=2.5
)

plt.title('The average discount changes over time', fontsize=14, fontweight='bold')
plt.xlabel('Order Date', fontsize=12)
plt.ylabel('Average discount', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

The average discount rate shows varying levels over time, possibly influenced by strategic promotions. Analyzing discount trends can help evaluate pricing effectiveness and customer responsiveness.

###  ---- 📊Profit changes over time ----

In [ ]:
plt.figure(figsize=(12, 5))
df_grouped_profit = df.groupby('Order Date')['Profit'].sum().reset_index()
sns.lineplot(
    data=df_grouped_profit,
    x='Order Date',
    y='Profit',
    color='#6ba368' 
)
plt.title('Profit changes over time')
plt.xlabel('Order Date')
plt.ylabel('Profit')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

- The graph shows the fluctuation of profits over time, which may indicate seasonal factors affecting sales and profitability.
- Periods of significant profit increases can be observed, often associated with sales seasons or high demand.
- Periods of significant decline may require further analysis to determine the cause (such as increased discounts or a decline in sales).

🧠 Insight: If months or periods with consistently high profits are identified, marketing campaigns can be enhanced during those periods to increase revenue. Weak periods, however, may require improvements to pricing or offering strategies.

## Analyzing the Impact of Order Timing on Segment, Product, and Region
In this section, we will examine how order timing (year, month, day of the week) affects:Different customer segments,
categories,
and geographic regions.

This helps us understand customer preferences and order timing by category, which supports improved targeting and logistics planning.

###  ---- 📊Effect of order time on segments ---- 

In [ ]:
plt.figure(figsize=(14,6))
sns.countplot(data=df, x='Order Month', hue='Segment', palette='Set2')
plt.title('Distribution of the number of orders by month and segment')
plt.xlabel('Order Month')
plt.ylabel('Number of Orders')
plt.legend(title='Segment')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

###  ---- 📊Impact of order time on categories ---- 

In [ ]:
plt.figure(figsize=(14,6))
top_categories = df['Category'].value_counts().index
sns.countplot(data=df[df['Category'].isin(top_categories)], x='Order Month', hue='Category', palette='Set1')
plt.title('Distribution of the number of ordres by month and category')
plt.xlabel('Order Month')
plt.ylabel('Number of Orders')
plt.legend(title='Category')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

###  ---- 📊Impact of demand time on regions ---- 

In [ ]:
plt.figure(figsize=(14,6))
sns.countplot(data=df, x='Order Month', hue='Region', palette='Paired')
plt.title('Distribution of the number of orders by month and region')
plt.xlabel('Order Month')
plt.ylabel('Number of Orders')
plt.legend(title='Region')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

## ===== Part Two: Analysis of Shipping Timing =====

### ---- 📊Analyze the most common shipping times for products ----

In [ ]:
plt.figure(figsize=(14, 10))
fig, axs = plt.subplots(2, 2, figsize=(16, 10))

# Number of shipments per year
sns.countplot(x='Ship Year', data=df, ax=axs[0, 0], hue='Ship Year', palette='Blues')
axs[0, 0].set_title('Number of shipments per year')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Number of shipments')

# Number of shipments per month
sns.countplot(x='Ship Month', data=df, ax=axs[0, 1], hue='Ship Month', palette='Greens')
axs[0, 1].set_title('Number of shipments per month')
axs[0, 1].set_xlabel('Month')
axs[0, 1].set_ylabel('Number of shipments')

# Number of shipments per day of the month
sns.countplot(x='Ship Day', data=df, ax=axs[1, 0], hue='Ship Day', palette='Oranges')
axs[1, 0].set_title('Number of shipments per day of the month')
axs[1, 0].set_xlabel('Day')
axs[1, 0].set_ylabel('Number of shipments')

# Number of shipments by day of the week
sns.countplot(x='Ship Weekday', data=df, 
              order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
              ax=axs[1, 1], hue='Ship Weekday', palette='Purples')
axs[1, 1].set_title('Number of shipments by day of the week')
axs[1, 1].set_xlabel('Day')
axs[1, 1].set_ylabel('Number of shipments')

plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insights:

The number of shipments increases a little every year, which may mean more customers.

Most shipments happen in November and December. This could be because of holidays.

There are more shipments in the middle of the month, around days 10 to 20.

Most shipments happen from Monday to Friday, and very few on weekends.

This helps plan better: more staff can work during busy times.



## Analyzing Shipping Date Relationships
This section also analyzes its temporal link with sales, profit, discounts, and quantity. The goal is to extract actionable logistics insights.

### ---- 📊Sales change by shipment date ----

In [ ]:
df_grouped_sales = df.groupby('Ship Date')['Sales'].sum().reset_index()
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_grouped_sales, x='Ship Date', y='Sales', color='navy')
plt.title('Sales change by shipment date')
plt.xlabel('Ship Date')
plt.ylabel('Total sales')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insight:
Sales peak strongly at year-end, likely driven by seasonal demand.

### ---- 📊Profit changes depending on shipping date ----

In [ ]:
df_grouped_profit = df.groupby('Ship Date')['Profit'].sum().reset_index()
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_grouped_profit, x='Ship Date', y='Profit', color='green')
plt.title('Profit changes depending on shipping date.')
plt.xlabel('Ship Date')
plt.ylabel('Profit')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insight: 
profit generally follows sales patterns, but variations suggest differences in cost or discounts.

### ---- 📊Discount changes depending on shipping date ----

In [ ]:
df_grouped_discount = df.groupby('Ship Date')['Discount'].mean().reset_index()
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_grouped_discount, x='Ship Date', y='Discount', color='purple')
plt.title('Discount changes depending on shipping date.')
plt.xlabel('Ship Date')
plt.ylabel('Average discount')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insight:
Discounts rise during specific periods, possibly due to promotions or end-of-season clearance.

### ---- 📊Quantity changes depending on shipping date ----

In [ ]:
df_grouped_quantity = df.groupby('Ship Date')['Quantity'].sum().reset_index()
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_grouped_quantity, x='Ship Date', y='Quantity', color='orange')
plt.title('Quantity changes depending on shipping date.')
plt.xlabel('Ship Date')
plt.ylabel('Total Quantity')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insight:
Quantities shipped tend to increase alongside sales, highlighting strong demand surges.

## Shipping Timing by Mode, Segment, and Region
This section analyzes how shipping timing (day of year) relates to shipping mode, customer segment, and region via boxplots. It also visualizes the evolution of key performance metrics (sales, profit, discount, quantity) over shipping dates.

### ----📊Distribution of shipping days by shipping mode ----

In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(x='Ship Mode', y=df['Ship Date'].dt.dayofyear, data=df, hue='Ship Mode', palette='Blues')
plt.title('Distribution of shipping days by shipping method')
plt.xlabel('Ship Mode')
plt.ylabel('Today of the year')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insight:
Second Class and Standard Class modes show broader shipping timing, whereas Same Day is faster and more consistent.

### ---- 📊Distribution of shipping days by segment ----

In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(x='Segment', y=df['Ship Date'].dt.dayofyear, data=df, hue='Segment', palette='Greens')
plt.title('Distribution of shipping days by segment')
plt.xlabel('Segment')
plt.ylabel('Today of the year')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insight:
Consumer segment shows a wide range of shipping throughout the year. Corporate and Home Office are more focused.

### ----📊 Distribution of shipping days by region ----

In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(x='Region', y=df['Ship Date'].dt.dayofyear, data=df, hue='Region', palette='Oranges')
plt.title('Distribution of shipping days by region')
plt.xlabel('Region')
plt.ylabel('Today of the year')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️Insight:
Western region has tighter shipping timing, while Southern region ships more broadly throughout the year.

## ===== Part Three: Shipping Performance and Operational Delays =====

### Shipping Delay Analysis by Month and Season
We analyzed the average number of days between the order date and the shipping date to identify if delays depend on the time of the year.

In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Order Season'] = df['Order Month'].apply(get_season)

# Calculate the average shipping delay
avg_delay_overall = df['Shipping Delay'].mean()

# Average calculation by month
avg_delay_by_month = df.groupby('Order Month')['Shipping Delay'].mean()

# Calculate the average by semester
avg_delay_by_season = df.groupby('Order Season')['Shipping Delay'].mean().reindex(['Winter', 'Spring', 'Summer', 'Fall'])

print(f"Average overall delay time: {avg_delay_overall:.2f} Days\n")
print("Average delay time by month:")
print(avg_delay_by_month)
print("\nAverage delay by semester:")
print(avg_delay_by_season)

### ---- 📊Average Delay by Month ----

In [ ]:
plt.figure(figsize=(7,3))
sns.lineplot(x=avg_delay_by_month.index, y=avg_delay_by_month.values, marker='o')
plt.title('Average shipping delay time by order month')
plt.xlabel('Month')
plt.ylabel('Average delay time (Days)')
plt.xticks(ticks=range(1,13))
plt.grid(True)
plt.show()

### ---- 📊Average Delay by Season ----

In [ ]:
plt.figure(figsize=(6,3))
sns.barplot(x=avg_delay_by_season.index, y=avg_delay_by_season.values, hue=avg_delay_by_season.index, palette='coolwarm')
plt.title('Average shipping delay by season')
plt.xlabel('Season')
plt.ylabel('Average delay time (Days)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

### Key Findings
Overall delay: Around 3 days on average.

By Month:

Delays are highest in January and April.

Some months (e.g., June, September) show quicker shipping.

By Season:

Winter has the highest average delay, possibly due to holidays or weather.

Summer and Fall tend to have slightly faster shipping.

#### 🕵️‍♂️Recommendations:
Inform customers during winter that slight delays may occur.

Investigate January and April for specific bottlenecks (e.g., supplier holidays, low staffing).

Use this data to plan logistics capacity better during high-delay months.

## ===== Part Four: Impact of Shipping Delays on Business Dimensions =====
- Different customer segments,
- Products and categories,
- Geographic areas (region, city),
- Ship Mode

The purpose is to identify the regions or categories experiencing the greatest shipping delays, to guide supply chain and customer service improvement efforts.

In [ ]:
# Filter data with positive delay only (ignore negative or zero values if they are illogical)
df_delay = df[df['Shipping Delay'] >= 0]

### ---- 📊Average shipping delay by Segment ----

In [ ]:
avg_delay_segment = df_delay.groupby('Segment')['Shipping Delay'].mean().round(2).sort_values()

plt.figure(figsize=(6,3))
sns.barplot(x=avg_delay_segment.values, y=avg_delay_segment.index, hue=avg_delay_segment.values, palette='viridis')
plt.title('Average shipping delay by Segment')
plt.xlabel('Average delay time (days)')
plt.ylabel('Segment')
plt.show()

### ---- 📊Average Shipping Delay by City - Top 15 Delayed Cities ----

In [ ]:
avg_delay_city = df_delay.groupby('City')['Shipping Delay'].mean().sort_values(ascending=False).head(15)

plt.figure(figsize=(10,4))
sns.barplot(x=avg_delay_city.values, y=avg_delay_city.index, hue=avg_delay_city.index, palette='cubehelix')
plt.title('Top 15 Cities with Average Shipping Delays')
plt.xlabel('Average delay time (days)')
plt.ylabel('City')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

### ---- 📊Average shipping delay by shipping method ----

In [ ]:
avg_delay_shipmode = df_delay.groupby('Ship Mode')['Shipping Delay'].mean().round(2).sort_values()

plt.figure(figsize=(6,3))
sns.barplot(x=avg_delay_shipmode.values, y=avg_delay_shipmode.index, hue=avg_delay_shipmode.values, palette='Set2')
plt.title('Average shipping delay by shipping mode')
plt.xlabel('Average delay time (days)')
plt.ylabel('Ship Mode')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

### ---- Print a quick summary of the results ----

In [ ]:
avg_delay_segment = df_delay.groupby('Segment')['Shipping Delay'].mean().round(2).sort_values()
avg_delay_category = df_delay.groupby('Category')['Shipping Delay'].mean().round(2).sort_values()
avg_delay_region = df_delay.groupby('Region')['Shipping Delay'].mean().round(2).sort_values()

print("Average shipping delay by Segment:\n", avg_delay_segment, "\n")
print("Average shipping delay by category:\n", avg_delay_category, "\n")
print("Average shipping delay by region:\n", avg_delay_region, "\n")
print("Average shipping delay by shipping mode:\n", avg_delay_shipmode, "\n")

#### 🕵️‍♂️ Summary of Findings
Segment:
Home Office customers face the longest delays.
Consumer orders tend to ship faster.

Top 15 Delayed Cities:
Certain urban centers consistently show higher shipping delays. This may be due to logistics complexity, distance from warehouses, or infrastructure issues.

Ship Mode:
Same Day mode has the shortest delays as expected.
Standard Class and Second Class modes introduce more variability and longer average delays.

Category:
Office Supplies tend to experience higher delays than Furniture or Technology.

#### 🕵️‍♂️ Recommendations
Investigate high-delay categories (like office supplies): Are there vendor or inventory issues?

Analyze city-level delays: Could traffic, distance, or courier service coverage be optimized?

Improve shipping method selection: Encourage faster methods for time-sensitive clients.

Educate customers: Provide accurate delivery time estimates depending on segment, city, and category.

Optimize logistics flow in cities or regions with frequent bottlenecks.

# 🔍 *Analysis by Segment*

## ===== Part One: Segment Behavior Analysis over Time =====

In this section, we will review:

The segments that purchase the most.
The evolution of orders, sales, and profit for each segment over time (monthly).

The goal of the analysis is to identify the most profitable or most in-demand segments during specific periods to guide marketing and inventory strategies.
Analyzing the impact of delays on various variables

### ---- 📊Most purchased segments (number of orders) ----

In [ ]:
plt.figure(figsize=(6,3))
sns.countplot(data=df, x='Segment', hue='Segment', palette='Set2', order=df['Segment'].value_counts().index)
plt.title('Number of orders by segment')
plt.xlabel('Segment')
plt.ylabel('Number of Orders')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🕵️‍♂️ Conclusions
The majority of orders come from the Consumer segment, followed by the Corporate segment. The Home Office segment has the lowest order volume, which could indicate an opportunity for targeted marketing or may reflect a smaller market size.

### ---- 📊Evolution of the number of Orders over time for each segment ----

In [ ]:
# Extract month and year name as time period
df['Year-Month'] = df['Order Date'].dt.to_period('M').astype(str)

orders_per_segment = df.groupby(['Year-Month', 'Segment'])['Order ID'].nunique().reset_index()
orders_per_segment.columns = ['Year-Month', 'Segment', 'Order Count']

plt.figure(figsize=(12,6))
sns.lineplot(data=orders_per_segment, x='Year-Month', y='Order Count', hue='Segment', marker='o')
plt.title('Number of monthly orders by segment')
plt.xlabel('Time (month)')
plt.ylabel('Number of orders')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

#### 🕵️‍♂️ Conclusions
The chart illustrates clear seasonality and differences in order volume across customer segments over time.

The Consumer segment consistently records the highest number of monthly orders.

The Corporate segment follows with moderate activity.

The Home Office segment shows the lowest and most stable order trend.
These trends highlight the importance of focusing marketing and inventory efforts on the Consumer segment, especially during peak months like November and December.



### ---- 📊Sales evolution over time for each segment ----

In [ ]:
sales_per_segment = df.groupby(['Year-Month', 'Segment'])['Sales'].sum().reset_index()

plt.figure(figsize=(12,6))
sns.lineplot(data=sales_per_segment, x='Year-Month', y='Sales', hue='Segment', marker='o')
plt.title('Monthly sales by segment')
plt.xlabel('Time (month)')
plt.ylabel('Total sales')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

### ---- 📊Profit evolution over time for each segment ----

In [ ]:
profit_per_segment = df.groupby(['Year-Month', 'Segment'])['Profit'].sum().reset_index()

plt.figure(figsize=(12,6))
sns.lineplot(data=profit_per_segment, x='Year-Month', y='Profit', hue='Segment', marker='o')
plt.title('Monthly profit by segment')
plt.xlabel('Time (month)')
plt.ylabel('Total profit')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

#### 🕵️‍♂️ Conclusions

- The segment with the highest purchasing power is: Consumer.
- There is a clear difference in order and sales volume between the three segments over time.
- Some segments have higher purchasing activity during certain months, which may indicate seasonal campaigns or cyclical purchasing behavior.
- The highest-selling segments are not necessarily the most profitable—the relationship between sales and profit should be monitored.
#### 🕵️‍♂️ Recommendations
- Focus on segments with high profitability, especially during the months when they are most active.
- Tailor marketing campaigns based on segment and time (for example, offering discounts to the consumer segment at the beginning of the year).
- Improve the customer experience for the segment with the highest profitability to maintain loyalty.


## ===== Part Two: The relationship between the segment and geographical locations =====
Are there clear geographic preferences for each customer segment? For example, is the "Corporate" segment more prevalent in certain states than the "Consumer" or "Home Office" segment? This type of analysis helps understand geographic differences and customize offers by segment and location.

### ---- 📊Distribution of segments by region ----

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Region', hue='Segment', palette='Set2')
plt.title('Distribution of segments by region')
plt.xlabel('Region')
plt.ylabel('Number of orders')
plt.grid(axis='y', alpha=0.4)
plt.legend(title='Segment')
plt.show()

### ---- 📊Distribution of segments by states (Top 10 only) ----

In [ ]:
top_states = df['State'].value_counts().head(10).index
plt.figure(figsize=(12, 6))
sns.countplot(data=df[df['State'].isin(top_states)], x='State', hue='Segment', palette='Set3')
plt.title('Distribution of segments in the top 10 states in terms of demand')
plt.xlabel('State')
plt.ylabel('Number of orders')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.4)
plt.legend(title='Segment')
plt.show()

### ---- 📊Distribution of segments by city (Top 10 only) ----

In [ ]:
top_cities = df['City'].value_counts().head(10).index
plt.figure(figsize=(12, 6))
sns.countplot(data=df[df['City'].isin(top_cities)], x='City', hue='Segment', palette='Pastel1')
plt.title('Distribution of segments in the top 10 cities in demand')
plt.xlabel('City')
plt.ylabel('Number of orders')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.legend(title='Segment')
plt.show()

#### 🕵️‍♂️ Conclusions

- We note that the "Consumer" segment represents the largest segment in most geographic regions, indicating that it is the largest target market.
- The "Corporate" segment has a significant presence in certain states, such as California and New York.
- A city like Los Angeles, New York, shows a greater balance between the three segments, which may indicate demographic and business diversity.

### ---- Average Profit and Sales Per Segment by Region ----
This analysis shows which customer segments generate the highest average profit or sales within each region. This can help us make strategic decisions about focusing on specific segments in specific regions.

### ---- 📊Average profit drawing ----

In [ ]:
# Collect data to calculate average profit and sales by segment and region.
profit_sales_by_segment_region = df.groupby(['Region', 'Segment'])[['Sales', 'Profit']].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=profit_sales_by_segment_region, x='Region', y='Profit', hue='Segment', palette='Set2')
plt.title('Average profit per segment by region')
plt.ylabel('Average profit')
plt.grid(axis='y')
plt.legend(title='Segment')
plt.show()

### ---- 📊Average sales chart ----

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=profit_sales_by_segment_region, x='Region', y='Sales', hue='Segment', palette='Set3')
plt.title('Average sales per segment by region')
plt.ylabel('Average sales')
plt.grid(axis='y')
plt.legend(title='Segment')
plt.show()

#### 🕵️‍♂️ Conclusions

In some regions, such as the West, the Corporate segment generates the highest average profit compared to the rest of the segments.

While in the South, the Consumer segment may generate the highest sales but lower profits, indicating higher discounts or lower profit margins.

## ===== Part Three: Relationship between segments, products, & categories ===== 
In this section, we focus on understanding how customer preferences for products and categories purchased vary across segments. This analysis helps us direct marketing, promotional, and inventory management strategies to suit each customer segment.

### ---- 📊Most purchased products per segment with profitability ----

In [ ]:
# Data collection by segment and product
product_summary = df.groupby(['Segment', 'Product Name']) \
                    .agg({'Sales': 'sum',
                          'Quantity': 'sum',
                          'Profit': 'sum'}) \
                    .reset_index()

# Calculate the profit margin for each product within each segment.
product_summary['Profit Margin %'] = (product_summary['Profit'] / product_summary['Sales']) * 100

# Show the top 10 products by sales for each segment.
top_products_per_segment = product_summary.sort_values(['Segment', 'Sales'], ascending=[True, False]) \
                                          .groupby('Segment') \
                                          .head(10)

plt.figure(figsize=(15,8))
sns.barplot(data=top_products_per_segment, y='Product Name', x='Sales', hue='Segment')
plt.title('Top 10 Products by Sales per Segment')
plt.xlabel('Total sales')
plt.ylabel('Product Name')
plt.legend(title='Segment')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🕵️‍♂️ Insights:

* Each segment has a distinct set of top-performing products, indicating varied preferences across customer groups.
* **Consumer segment** features products with consistently high sales, showing strong market penetration.
* Some products appear across multiple segments, suggesting they have broad appeal and could benefit from cross-segment promotions.
* Understanding which products drive sales within each segment helps optimize inventory, marketing, and customer targeting strategies.


### ---- 📊Relationship between the discount and sales for each segment ----

In [ ]:
# Group sales by segment and discount
discount_impact = df.groupby(['Segment', 'Discount']).agg({
    'Sales': 'sum',
    'Quantity': 'sum',
    'Profit': 'sum'
}).reset_index()

# Graph the relationship between the discount and sales for each segment
plt.figure(figsize=(14, 6))
sns.lineplot(data=discount_impact, x='Discount', y='Sales', hue='Segment', marker='o')
plt.title('Impact of discounts on sales by segment')
plt.xlabel('نسبة الخصم')
plt.ylabel('إجمالي المبيعات')
plt.grid(True)
plt.show()

#### 🕵️‍♂️ Conclusions
Discount effectiveness differs by segment. Moderate discounts improve sales, but excessive discounts may not yield better results.

### ---- 📊Relationship between segment and shipping method ----

In [ ]:
shipping_pref = df.groupby(['Segment', 'Ship Mode']).agg({'Order ID': 'nunique'}).reset_index()
shipping_pref.rename(columns={'Order ID': 'Order Count'}, inplace=True)

plt.figure(figsize=(10,6))
sns.barplot(data=shipping_pref, x='Segment', y='Order Count', hue='Ship Mode')
plt.title('Relationship between segment and shipping method')
plt.xlabel('Segment')
plt.ylabel('Number of Orders')
plt.legend(title='Ship Mode')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🕵️‍♂️ Conclusions
Different segments show distinct shipping preferences. Standard shipping dominates, but premium options are preferred by some segments.

### ---- 📊Average sales per segment by month ----

In [ ]:
avg_sales_segment_month = df.groupby(['Order Month', 'Segment'])['Sales'].mean().unstack()

plt.figure(figsize=(14,6))
sns.heatmap(avg_sales_segment_month, annot=True, fmt=".0f", cmap='YlGnBu')
plt.title('Average sales per segment by month ')
plt.xlabel('Segment')
plt.ylabel('Order Month')
plt.show()

#### 🕵️‍♂️ Conclusions

- The graphs show clear differences in the timing of orders between different segments, with some segments tending to purchase more during certain months.
- Categories and products vary in demand over time, reflecting that some products are seasonal or associated with specific periods.
- Demand across geographic areas varies monthly, indicating local or seasonal factors that influence purchasing behavior.
- Average sales per segment indicate which months are most profitable for each customer segment.


### ---- 📊Heatmap of Total Sales by Region and Product Category ----

In [ ]:
# Pivot table: total sales by Region and Category
pivot_region_category = df.pivot_table(index='Region', columns='Category', values='Sales', aggfunc='sum')

plt.figure(figsize=(10, 7))
sns.heatmap(pivot_region_category, annot=True, fmt='.0f', cmap='YlGnBu')
plt.title('Heatmap of Sales by Region and Category')
plt.xlabel('Category')
plt.ylabel('Region')
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions 

This heatmap shows the total sales distribution across different regions and product categories.

The West and East regions lead in overall sales, particularly in the Technology and Office Supplies categories.

Furniture performs relatively better in the South and Central regions than Technology.

Technology sales are notably strong in the West region.
These insights suggest tailoring product offerings and promotions by region—for example, focusing more on Technology in the West and Furniture in the South.


### ---- 📊Interactive Choropleth Map: Order Distribution by Segment and State ----
The interactive map displays the number of applications for each segment in each state, allowing us to visualize where different segments are concentrated in the United States.

In [ ]:
# Map converting state names to their abbreviated codes
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR',
    'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Calculate the number of orders for each segment in each state
orders_by_state_segment = df.groupby(['State', 'Segment'])['Order ID'].nunique().reset_index()
orders_by_state_segment.columns = ['State', 'Segment', 'Order Count']

# Add short state code
orders_by_state_segment['State Code'] = orders_by_state_segment['State'].map(state_abbrev)

# Delete rows for which no symbol was found
orders_by_state_segment.dropna(subset=['State Code'], inplace=True)

# Interactive map drawing
fig = px.choropleth(
    orders_by_state_segment,
    locations='State Code',
    locationmode='USA-states',
    color='Order Count',
    scope='usa',
    animation_frame='Segment',
    hover_name='State',
    labels={'Order Count': 'Number of orders'},
    color_continuous_scale='Viridis',
    title='Distribution of the number of orders by segment in each state (interactive)'
)

fig.update_layout(geo=dict(bgcolor='rgba(0,0,0,0)'), title_x=0.5)
fig.show()

#### 🕵️‍♂️ Insight:

The interactive map reveals geographic differences in order distribution across customer segments.

States like California, New York, and Texas show high order volumes across all segments, indicating strong market presence and customer engagement.

Less populated or rural states, especially in the Midwest, show lower volumes—particularly in the Corporate segment.

The animation by segment highlights regional preferences for different customer types, such as more Consumer and Home Office activity in the West and East coasts.
This visualization supports region-specific marketing and expansion strategies, helping businesses align their offerings with geographic demand patterns.



# 🔍 *Geographical analysis*

### ---- 📊Top 10 cities by sales ----

In [ ]:
# Group data by country, region, state, and city
region_perf = df.groupby('Region').agg({
    'Order ID': 'nunique',
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'mean',
    'Discount': 'mean'
}).reset_index()

region_perf.columns = ['Region', 'Order Count', 'Total Sales', 'Total Profit', 'Avg Quantity', 'Avg Discount']

region_perf = df.groupby(['Country', 'Region', 'State', 'City'])[['Sales', 'Profit']].sum().reset_index()

# Top 10 cities by sales
top_sales_cities = region_perf.sort_values(by='Sales', ascending=False).head(10)
plt.figure(figsize=(12, 6))
sns.barplot(data=top_sales_cities, y='City', x='Sales', hue='Sales', palette='Blues_d')
plt.title('Top 10 cities by sales')
plt.xlabel('Total Sales')
plt.ylabel('City')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

#### 🕵️‍♂️ Conclusions 
The top 10 cities by sales contribute significantly to the overall revenue, indicating strong local market presence. Targeting marketing and inventory strategies in these high-performing cities could enhance profitability and customer engagement further.


### ---- 📊Top-Profitable States ----

In [ ]:
top_profit_states = region_perf.groupby('State').sum(numeric_only=True).sort_values('Profit', ascending=False).head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_profit_states['Profit'], y=top_profit_states.index, hue=top_profit_states['Profit'], palette='Greens_d')
plt.title('Top 10 states by profit')
plt.xlabel('Total Profit')
plt.ylabel('State')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

#### 🕵️‍♂️ Conclusions 
The top 10 most profitable states significantly impact the overall business profitability. These states likely have a healthy balance of sales volume and efficient cost management. Focusing on maintaining and expanding operations in these regions could further boost profit margins.

### ---- 📊Distribution of Quantity by Region ----

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxenplot(data=df, x='Region', y='Quantity', hue='Region', palette='Set2')
plt.title('Quantity distribution analysis by region')
plt.ylabel('Quantity')
plt.xlabel('Region')
plt.grid(axis='y')
plt.show()

### ---- 📊Interactive Bubble Plot: Sales vs Quantity by Region with Profit Size ----

This interactive scatter plot visualizes the relationship between total quantity and total sales across different regions. The bubble size represents the total profit, giving an intuitive view of overall regional performance.


In [ ]:
# Summary data for each region
region_summary = df.groupby('Region').agg({
    'Sales': 'sum',
    'Quantity': 'sum',
    'Profit': 'sum'
}).reset_index()

# Interactive bubble plot
fig = px.scatter(
    region_summary,
    x='Quantity',
    y='Sales',
    size='Profit',
    color='Region',
    hover_name='Region',
    title='Relationship between Quantity and Sales by Region (Bubble Size = Profit)',
    labels={'Sales': 'Total Sales', 'Quantity': 'Total Quantity'}
)
fig.show()

#### 🕵️‍♂️ Insights:

- Regions like **West** and **East** exhibit higher total sales and quantities, along with larger profit bubbles, suggesting they are the most commercially successful regions.
- **Central** shows moderate sales but relatively smaller profits, which may indicate lower profit margins.
- **South** has the smallest bubble, indicating lower sales and profits, possibly suggesting an opportunity for growth or underperformance.
- The plot helps identify efficient regions (high profit with lower quantity) vs. volume-driven regions (high quantity but low margin).


### ---- 📊Distribution of Shipping Methods by Region ----
This count plot displays the number of orders using each shipping method across different regions. It helps understand how logistics strategies vary by region.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Region', hue='Ship Mode')
plt.title('Shipping Methods Distribution by Region')
plt.xlabel('Region')
plt.ylabel('Number of Orders')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insights:
- **Standard Class** dominates as the most used shipping method in all regions, indicating it is the default or preferred option.
- **Same Day** shipping is rarely used and appears mostly in highly urbanized regions like the **West**.
- **First Class** is more common in the **West** and **East**, suggesting a higher demand or capability for faster deliveries.
- The **Central** region has the lowest usage of **Same Day** and **First Class**, possibly due to infrastructure or demand factors.

### ---- 📊Shipping Delay Analysis by Region ----
This plot visualizes the number of orders per region separated by delay status, where delayed shipments are defined as those with shipping delay exceeding the median value.

In [ ]:
# Define delayed shipments: Shipping Delay greater than median shipping delay
median_delay = df['Shipping Delay'].median()
df['Delayed'] = df['Shipping Delay'] > median_delay

# Plot countplot of orders by region colored by delay status
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Region', hue='Delayed', palette='Set1')
plt.title('Count of Orders by Region and Delay Status')
plt.xlabel('Region')
plt.ylabel('Number of Orders')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insights:

- The plot highlights regions with higher counts of delayed shipments, suggesting potential logistical or operational bottlenecks in these areas.
- Regions with larger shares of delayed orders may benefit from focused process improvements or enhanced shipping strategies.
- Combining delay data with shipping mode and other variables could reveal more precise causes and solutions for delays.


### ---- 📊Closest Warehouse Location Estimation Based on Average Shipping Delay ----
This analysis estimates the most probable warehouse location by identifying the state with the shortest average shipping delay. The interactive map visualizes average shipping delays across states, aiding spatial understanding of shipping efficiency.

In [ ]:
# Calculate shipping delay in days (ensure 'Order Date' and 'Ship Date' are datetime)
df['Shipping Delay (Days)'] = (df['Ship Date'] - df['Order Date']).dt.days

# Calculate average shipping delay per state
state_shipping = df.groupby('State')['Shipping Delay (Days)'].mean().reset_index()
state_shipping.columns = ['State', 'Avg Shipping Delay']

# Approximate latitude and longitude for selected US states
us_states_coords = pd.DataFrame({
    'State': ["California", "New York", "Texas", "Illinois", "Washington", "Florida", "Pennsylvania", "Ohio", "Colorado", "Georgia"],
    'Latitude': [36.7783, 40.7128, 31.9686, 40.6331, 47.7511, 27.9944, 41.2033, 40.4173, 39.5501, 32.1656],
    'Longitude': [-119.4179, -74.0060, -99.9018, -89.3985, -120.7401, -81.7603, -77.1945, -82.9071, -105.7821, -82.9001]
})

# Merge average delay data with state coordinates
merged = pd.merge(state_shipping, us_states_coords, on='State', how='left')

# Drop states without coordinates
merged = merged.dropna(subset=['Latitude', 'Longitude'])

# Identify the closest city (state with minimum average shipping delay)
closest_city = merged.loc[merged['Avg Shipping Delay'].idxmin()]
print("\n Closest probable warehouse location: ")
print(f"- {closest_city['State']} (Average Delay: {closest_city['Avg Shipping Delay']:.2f} days)")

# Interactive map showing average shipping delay by state
fig = px.scatter_geo(
    merged,
    lat='Latitude',
    lon='Longitude',
    color='Avg Shipping Delay',
    text='State',
    size='Avg Shipping Delay',
    color_continuous_scale='YlOrRd_r',
    scope='usa',
    title='Average Shipping Delay by State (Closest indicates warehouse location)'
)
fig.update_layout(geo=dict(projection_scale=3.5, center={"lat": 37.5, "lon": -95}))
fig.show()


#### 🕵️‍♂️ Insights:

The analysis estimates the probable store location by identifying the state with the shortest average shipping delay. This approach assumes that shipments originating closer to the store will have shorter delivery times. According to the results, the state with the lowest average shipping delay is most likely to house the store or main distribution center.

However, this inference should be interpreted cautiously. Shipping delay depends on multiple factors beyond geographic proximity, such as logistics infrastructure, shipping methods, order volume, and carrier networks. Therefore, while the identified state is a strong candidate for the store location, it is not definitive proof.

This method serves as an initial exploratory step to narrow down potential locations, which can be further refined by integrating additional operational and logistic data.


### ---- 📊Standard Deviation of Shipping Delay by State ----

In [ ]:
# Calculate mean and std deviation of shipping delay per state
state_stats = df.groupby('State')['Shipping Delay'].agg(['mean', 'std']).sort_values('mean')

plt.figure(figsize=(12, 6))
sns.barplot(data=state_stats.reset_index(), x='State', y='std', hue='State', palette='coolwarm')
plt.xticks(rotation=90)
plt.title('Standard Deviation of Shipping Delay by State')
plt.ylabel('Standard Deviation (Days)')
plt.xlabel('State')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


### ---- 📊Category Distribution Across Regions ----

In [ ]:
# Count of orders per category in each region
category_region = df.groupby(['Region', 'Category']).size().reset_index(name='Count')

# Create stacked bar chart using Plotly
fig = px.bar(
    category_region,
    x='Region',
    y='Count',
    color='Category',
    title='Category Distribution by Region',
    barmode='stack',
    text='Count'
)

fig.update_traces(textposition='inside')
fig.update_layout(xaxis_title='Region', yaxis_title='Count')
fig.show()


# 🔍 *Analysis by Product*

### ---- 📊Top 15 Best-Selling Products Analysis ----

In [ ]:
top_products = df.groupby('Product Name').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum',
    'Profit Margin': 'mean',
    'Unit Price': 'mean'
}).sort_values(by='Sales', ascending=False).head(15).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(data=top_products, y='Product Name', x='Sales', hue='Sales', palette='Blues_d')
plt.title('Top 15 Best-Selling Products')
plt.xlabel('Total Sales')
plt.ylabel('Product Name')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


#### 🕵️‍♂️ Insights:
“When you have eliminated the impossible, whatever remains, however improbable, must be the truth.”

High sales ≠ High profitability
A product that dominates the sales chart might still suffer from poor profit margins. Sales alone can be misleading if not balanced with profitability metrics.

Product concentration may be risky
A heavy dependence on a few products for revenue introduces vulnerability. A sudden dip in any of these products’ performance could ripple across the business.

Unit price and volume trade-off
Some top-selling products might be priced low and rely on high volume. This can work—but only if operational costs remain controlled.

Room for strategic repricing or bundling
If a top-selling item has low profit margins, it might benefit from price optimization or being part of a value bundle to boost profitability.


### ---- 📊Comparative Analysis: Top-Selling vs. Top-Profitable Products ----

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')

top_sales = df.groupby('Product Name')["Sales"].sum().sort_values(ascending=False).head(10)
top_profit = df.groupby('Product Name')["Profit"].sum().sort_values(ascending=False).head(10)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.barplot(x=top_sales.values, y=top_sales.index, ax=axes[0])
axes[0].set_title('Top Selling Products')
axes[0].set_xlabel('Total Sales')

sns.barplot(x=top_profit.values, y=top_profit.index, ax=axes[1])
axes[1].set_title('Top Profitable Products')
axes[1].set_xlabel('Total Profit')

plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insights:
“The curious case is not in what is visible to the naked eye, but in the absence of overlap between success and profitability.”

Not all that sells is gold.
A product may dominate in total revenue but not appear among the most profitable — revealing high costs or deep discounting strategies.

Profit leaders are often stealthy.
Some products with modest sales volumes yield significantly higher profit margins, suggesting they may be premium or cost-efficient.

Operational mystery: margin leakages.
When top-selling products fail to generate matching profits, it’s a signal to investigate cost structures, shipping overhead, or post-sale services.

Strategic clarity lies in contrast.
This dual-panel view helps distinguish between volume-driven and margin-driven success, enabling better product strategy and portfolio alignment.

“What one must notice is not merely what products are present, but which ones are conspicuously absent between the charts. That, my friend, is where the game begins.”

### ---- 📊Regional Leaders: Top 3 Products by Quantity Sold in Each Region ----

In [ ]:
region_product = df.groupby(['Region', 'Product Name'])['Quantity'].sum().reset_index()
top_products_region = region_product.sort_values(['Region', 'Quantity'], ascending=[True, False]).groupby('Region').head(3)

plt.figure(figsize=(14, 6))
sns.barplot(data=top_products_region, x='Region', y='Quantity', hue='Product Name')
plt.title('Top 3 Products by Quantity Sold per Region')
plt.ylabel('Total Quantity')
plt.xlabel('Region')
plt.legend(title='Product Name')
plt.show()

#### 🕵️‍♂️ Insights:
“When investigating regional behavior, one must always ask not what is common everywhere — but what is exceptionally common somewhere.”

Regional preferences are not uniform.
The top 3 best-selling products vary significantly across regions, suggesting that customer needs and priorities shift with geography.

Product success is context-bound.
A product that dominates in one region may not even appear in another — pointing to the influence of local demographics, climates, or supply chains.

A mirror of demand hotspots.
The visualization acts as a magnifying glass to uncover which products form the cornerstone of regional sales — essential for tailored marketing or logistics.

Strategic implication: regional specialization.
Optimizing stock, promotions, and delivery routes around these high-demand products can reduce costs and boost regional performance.

“It is elementary: the product most favored by each land whispers the tale of its people — their habits, their lifestyle, their needs.”

### ---- 📊Evolution of Sales Volume Over Time for Top 5 Products ----

In [ ]:
# Aggregate total quantity sold per product over time
product_quantity_over_time = df.groupby(['Order Date', 'Product Name'])['Quantity'].sum().reset_index()

# Select top 5 most frequently sold products
top_products = df['Product Name'].value_counts().nlargest(5).index
filtered_data = product_quantity_over_time[product_quantity_over_time['Product Name'].isin(top_products)]

# Plot line chart showing quantity trends over time
plt.figure(figsize=(14, 7))
sns.lineplot(data=filtered_data, x='Order Date', y='Quantity', hue='Product Name')
plt.title('Total Quantity Sold Over Time for Top 5 Products')
plt.xlabel('Order Date')
plt.ylabel('Quantity Sold')
plt.legend(title='Product Name')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insights:
Demand is not static; it evolves.
The sales quantity of the top 5 products fluctuates visibly across time, indicating seasonality or periodic spikes driven by marketing, holidays, or external events.

Temporal patterns reveal strategy.
Peaks in demand for certain products suggest critical sales periods — valuable information for inventory planning and targeted promotions.

Each product tells a story.
While some maintain consistent demand, others show bursts — outliers that may signal successful campaigns or urgent market needs.

The unexpected must not be ignored.
A sudden drop or surge can be a clue — either a success to replicate or a problem to solve.

“The secret to foresight lies in understanding the rhythms of the past.”

### ---- 📊Sales Trends of the Top 10 Products Over Time ----

In [ ]:
# Select top 10 best-selling products based on total sales
top_products = df.groupby('Product Name')['Sales'].sum().nlargest(10).index

# Filter the dataset for only these products
filtered_df = df[df['Product Name'].isin(top_products)]

# Group sales by product and order date
sales_trend = filtered_df.groupby(['Order Date', 'Product Name'])['Sales'].sum().reset_index()

# Plot the sales trends over time for top-selling products
plt.figure(figsize=(14, 7))
sns.lineplot(data=sales_trend, x='Order Date', y='Sales', hue='Product Name')
plt.title('Sales Trends Over Time for Top 10 Best-Selling Products')
plt.xlabel('Order Date')
plt.ylabel('Total Sales')
plt.legend(title='Product Name', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insights:
The chart reveals not just what sells the most, but when it sells.
This temporal insight is critical for demand forecasting, promotional planning, and stock management.

Some products show consistent popularity over time, while others demonstrate sharp spikes.
These spikes might indicate successful short-term promotions or seasonal demand.

Tracking monetary sales rather than quantity provides a revenue-centric view.
High sales might come from fewer units with high unit prices, or large volumes with lower prices.

This time-series view bridges tactical decisions (daily sales) with strategic planning (long-term growth).

"Data doesn't just tell us what happened — it whispers why it happened, if we care to listen closely."


# 🔍 *Analysis by Shipping Mode*

### ---- 📊Most Frequently Used Shipping Method ----

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(
    data=df,
    x='Ship Mode',
    order=df['Ship Mode'].value_counts().index,
    hue='Ship Mode',
    palette='pastel'
)
plt.title('Most Frequently Used Shipping Method', fontsize=14, fontweight='bold')
plt.xlabel('Shipping Method', fontsize=12)
plt.ylabel('Number of Orders', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Insights:

Standard Class dominates the shipping preferences, likely due to its balance between cost and delivery time.

The popularity of this shipping method suggests it might be the default option selected by customers or the one promoted most during checkout.

Less frequently used methods such as Same Day or First Class may indicate higher cost, limited availability, or low perceived value by customers.

However, frequency of use does not always equate to effectiveness. If previous shipping delay analyses showed high delay rates for the most used method, this could point to systemic inefficiencies or customer compromise due to price.

Businesses should consider cross-analyzing this preference with shipping delays, profit margins, and customer satisfaction metrics to ensure the chosen method truly serves both the company and the customers effectively.


## ===== Analysis of the relationship between the Ship Mode & related variables =====
We'll review how shipping methods affect:
- Segment
- Region
- Category
- Shipping Delay
- Sales
- Profit

### ---- 📊Shipping Mode Distribution by Segment ----

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='Ship Mode', hue='Segment', palette='Set3')
plt.title('Distribution of Shipping Modes by Segment')
plt.xlabel('Shipping Mode')
plt.ylabel('Number of Orders')
plt.legend(title='Segment')
plt.show()

#### 🕵️‍♂️ Insights:
Consumer and Corporate segments tend to favor Standard Class and Second Class, suggesting that cost-effectiveness is a major priority for these groups.

Home Office customers, while fewer in number, show a relatively higher usage of First Class, possibly due to smaller but more urgent orders.

This distribution implies that customer priorities (speed vs. price) differ by segment, and the shipping strategy should be tailored accordingly.


### ---- 📊Analysis of Shipping Mode Distribution by Region ----

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='Ship Mode', hue='Region', palette='Pastel1')
plt.title('Distribution of Shipping Modes by Region')
plt.xlabel('Shipping Mode')
plt.ylabel('Number of Orders')
plt.legend(title='Region')
plt.show()

#### 🕵️‍♂️ Insights:
The West and East regions show a higher reliance on Standard Class, suggesting either longer shipping distances (making faster options costly) or customer willingness to wait.

The South region shows a relatively balanced usage of all shipping modes, which could indicate either a well-optimized logistics network or varied customer profiles.

Interestingly, the Central region shows a notable spike in Second Class and First Class usage, which might suggest:

A logistical hub in that region, allowing quicker fulfillment,

Or customers expecting faster deliveries due to proximity to distribution centers.

### ---- 📊Analysis of Shipping Mode Distribution by Product Category ----

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='Ship Mode', hue='Category', palette='Accent')
plt.title('Distribution of Shipping Modes by Product Category')
plt.xlabel('Shipping Mode')
plt.ylabel('Number of Orders')
plt.legend(title='Category')
plt.show()

#### 🕵️‍♂️ Insights:
The Technology category shows a higher share of Second Class and First Class shipping, likely due to:

Higher product value, justifying faster shipping.

Sensitivity to delivery times—customers often expect electronics to arrive promptly.

The Office Supplies category is predominantly shipped via Standard Class, indicating:

Lower urgency or value per item.

Bulk or routine purchasing patterns by businesses.

For Furniture, shipping appears evenly distributed, with notable use of Standard Class, possibly due to:

Size and weight making expedited shipping costly.

Longer expected lead times already accepted by customers.

### ---- 📊Average Shipping Delay by Shipping Mode ----

In [ ]:
avg_delay_shipmode = df.groupby('Ship Mode')['Shipping Delay'].mean()

plt.figure(figsize=(8,4))
sns.barplot(x=avg_delay_shipmode.index, y=avg_delay_shipmode.values, hue=avg_delay_shipmode.index, palette='Set2')
plt.title('Average Shipping Delay by Shipping Mode')
plt.xlabel('Shipping Mode')
plt.ylabel('Average Shipping Delay (Days)')
plt.grid(True, linestyle='--', alpha=0.36)
plt.show()

#### 🕵️‍♂️ Conclusions
Standard Class consistently shows the highest average delay, which aligns with its economical and non-expedited nature.

Same Day and First Class modes demonstrate significantly lower delay averages, suggesting:

Stronger logistic prioritization.

Possibly better coordination with local/regional warehouses.

Second Class sits in the middle, offering a balance between cost and timeliness.

This distribution is not random; it's a reflection of deliberate logistical strategies. Where speed is not paid for, it is not guaranteed — a truth evident in the delay data.

### ---- 📊Average Sales by Shipping Mode ----

In [ ]:
avg_sales_shipmode = df.groupby('Ship Mode')['Sales'].mean()

plt.figure(figsize=(8,4))
sns.barplot(x=avg_sales_shipmode.index, y=avg_sales_shipmode.values, hue=avg_sales_shipmode.index, palette='Blues_d')
plt.title('Average Sales by Shipping Mode')
plt.xlabel('Shipping Mode')
plt.ylabel('Average Sales')
plt.grid(True, linestyle='--', alpha=0.1)
plt.show()

#### 🕵️‍♂️ Conclusions

First Class and Same Day modes register higher average sales per order, indicating that:

Premium shipping options are more commonly selected for higher-value orders.

Customers may be willing to pay more when urgency or quality is involved.

In contrast, Standard Class—the most affordable option—correlates with lower sales per order, suggesting:

It's more frequently used for smaller, less expensive purchases.

Possibly favored by budget-conscious or non-urgent customers.

The link between order value and delivery speed is not coincidental. High sales often justify faster logistics, making the shipping mode a proxy for customer priority and product value.

### ---- 📊Average Profit by Shipping Mode ----

In [ ]:
avg_profit_shipmode = df.groupby('Ship Mode')['Profit'].mean()

plt.figure(figsize=(8,4))
sns.barplot(x=avg_profit_shipmode.index, y=avg_profit_shipmode.values, hue=avg_profit_shipmode.index, palette='Greens_d')
plt.title('Average Profit by Shipping Mode')
plt.xlabel('Shipping Mode')
plt.ylabel('Average Profit')
plt.grid(True, linestyle='--', alpha=0.1)
plt.show()

#### 🕵️‍♂️ Conclusions

First Class and Same Day show higher average profit per order, reinforcing their role in premium transactions:

These modes likely involve higher-margin items or customers more tolerant to price premiums.

Their operational cost might be higher, but the per-order profit remains attractive.

Standard Class yields the lowest average profit, possibly due to:

Smaller order sizes.

Lower product margins.

Price-sensitive segments.

Profit behavior aligns closely with sales findings—fast and premium shipping is often paired with high-value, high-margin transactions. This reinforces the idea that optimizing these channels could yield significant gains.

### ---- 📊Summary Statistics by Shipping Mode ----

In [ ]:
# Print a quick summary of the averages
print("Average Shipping Delay by Shipping Mode:\n", avg_delay_shipmode, "\n")
print("Average Sales by Shipping Mode:\n", avg_sales_shipmode, "\n")
print("Average Profit by Shipping Mode:\n", avg_profit_shipmode, "\n")

#### 🕵️‍♂️ Conclusions 
"It is a capital mistake to theorize before one has data." 

This quick summary confirms earlier visual findings with exact numbers:

Same Day shipping typically has the lowest delay and highest profit, making it efficient yet lucrative.

Standard Class shipping exhibits longer delays and lower profits, hinting at potential inefficiencies.

First Class often balances high sales and good profitability, possibly appealing to high-value customer segments.

These metrics reinforce the importance of aligning shipping strategies with customer expectations and profitability targets.

### ---- 📊Shipping Mode Distribution by Product Sub-Category ----

In [ ]:
plt.figure(figsize=(14, 6))
sns.countplot(data=df, y='Sub-Category', hue='Ship Mode', order=df['Sub-Category'].value_counts().index)
plt.title('Shipping Mode Distribution by Product Sub-Category')
plt.xlabel('Number of Orders')
plt.ylabel('Sub-Category')
plt.legend(title='Ship Mode')
plt.grid(True, linestyle='--', alpha=0.2)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions
This chart explores the distribution of shipping modes across product sub-categories. We observe that certain categories like "Paper" or "Binders" may favor standard or same-day shipping, while more delicate or high-value items like "Phones" or "Chairs" may lean toward expedited or first-class shipping.
A sub-category with unusual shipping preferences could hint at operational bottlenecks, special handling requirements, or opportunities to optimize shipping costs.

# 🔍 *Financial & Sales Performance Analysis*

### ---- 📊Total Sales, Profit, and Discounts by Segment ----

In [ ]:
aggs = df.groupby(['Segment','Region', 'Category'])[['Sales', 'Profit', 'Discount']].sum().reset_index()
aggs_melted = aggs.melt(id_vars=['Segment','Region', 'Category'], 
                        value_vars=['Sales', 'Profit', 'Discount'], 
                        var_name='Metric', 
                        value_name='Value')

plt.figure(figsize=(14, 6))
sns.barplot(data=aggs_melted, x='Segment', y='Value', hue='Metric', palette='Set1')
plt.title('Total Sales, Profit, and Discount by Segment')
plt.ylabel('Value')
plt.xlabel('Segment')
plt.legend(title='Metric')
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions
The Consumer segment leads in total sales and profit but also shows high discounts. This may reflect a volume-driven approach with moderate margins. The Corporate segment performs well with balanced metrics, while Home Office lags behind in overall financial contribution.


### ---- 📊Total Sales, Profit, and Discounts by Region ----

In [ ]:
plt.figure(figsize=(14, 6))
sns.barplot(data=aggs_melted, x='Region', y='Value', hue='Metric', palette='Set2')
plt.title('Total Sales, Profit, and Discount by Region')
plt.ylabel('Value')
plt.xlabel('Region')
plt.legend(title='Metric')
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions
The West and East regions contribute the most to both sales and profit, indicating high customer demand. The South region has lower performance but similar discount levels, suggesting room for marketing or pricing strategy improvements.

### ---- 📊Total Sales, Profit, and Discounts by Category ----

In [ ]:
plt.figure(figsize=(14, 6))
sns.barplot(data=aggs_melted, x='Category', y='Value', hue='Metric', palette='Set3')
plt.title('Total Sales, Profit, and Discount by Category')
plt.ylabel('Value')
plt.xlabel('Category')
plt.legend(title='Metric')
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions
Technology products generate the highest sales and profit with moderate discount levels, making them the most financially rewarding category. Furniture shows significant discounts but relatively lower profit, which may indicate either over-discounting or low margins. Office Supplies has steady sales but lower profitability.

### ---- 📊Average Net Profit Margin by Category ----

In [ ]:
df['Net Profit Margin'] = (df['Profit'] / (df['Sales'] + 1e-6)) * 100

margin_category = df.groupby('Category')['Net Profit Margin'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=margin_category, x='Category', y='Net Profit Margin')
plt.title('Average Net Profit Margin by Category')
plt.ylabel('% Net Profit Margin')
plt.xlabel('Category')
plt.show()

#### 🕵️‍♂️ Conclusions
“While all categories bring in revenue, not all are equally rewarding.”
The Technology category shows a significantly higher average net profit margin, indicating not only strong revenue but also lean costs or minimal discounts. In contrast, Furniture and Office Supplies exhibit much thinner margins, which could suggest either higher logistics costs, price sensitivity, or frequent discounting tactics.
This signals that not all sales are created equal — Technology is likely the most strategically lucrative.


### ---- 📊Top 10 Products by Net Profit Margin ---- 

In [ ]:
margin_product = df.groupby('Product Name')['Net Profit Margin'].mean().sort_values(ascending=False).head(10).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(data=margin_product, x='Net Profit Margin', y='Product Name')
plt.title('Top 10 Products by Net Profit Margin')
plt.xlabel('% Net Profit Margin')
plt.ylabel('Product Name')
plt.show()

#### 🕵️‍♂️ Conclusions
“The crown jewels of profitability are often not the most visible.”
These top 10 products aren't necessarily the best-selling ones, but they quietly deliver the highest returns per dollar of revenue. Such products may involve premium pricing, strong brand loyalty, or low competition.
They are the hidden champions of the catalog, and should be strategically promoted, prioritized in procurement, and potentially used as models for expanding the high-margin portfolio.


### ---- 📊 Net Profit Margin by Region ---- 

In [ ]:
margin_region = df.groupby('Region')['Net Profit Margin'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=margin_region, x='Region', y='Net Profit Margin')
plt.title('Net Profit Margin by Region')
plt.ylabel('% Net Profit Margin')
plt.xlabel('Region')
plt.show()

#### 🕵️‍♂️ Conclusions
“Profitability doesn’t just depend on what you sell—but also where you sell it.”
The West region leads in profit margins, revealing either lower operational costs, stronger purchasing power, or less discount-driven sales behavior.
Meanwhile, Central and South regions trail behind, which could hint at inefficiencies, price sensitivity, or logistical challenges. This opens a case for region-specific pricing strategies and targeted efficiency improvements in underperforming areas.

### ---- 📊Effect of Discount Brackets on Average Profit ----

In [ ]:
# Group discounts into intervals
discount_bins = pd.cut(df['Discount'], bins=[0, 0.1, 0.2, 0.3, 0.4, 1], labels=['0-10%', '10-20%', '20-30%', '30-40%', '>40%'])
discount_profit = df.groupby(discount_bins, observed=False)[['Profit']].mean().reset_index()
# Plot average profit by discount range
plt.figure(figsize=(10, 6))
sns.barplot(data=discount_profit, x='Discount', y='Profit', hue='Discount', palette='coolwarm')
plt.title('Impact of Discount on Average Profit')
plt.ylabel('Average Profit')
plt.xlabel('Discount Range')
for i, val in enumerate(discount_profit['Profit']):
    plt.text(i, val, f'{val:.2f}', ha='center', va='bottom')
plt.show()


#### 🕵️‍♂️ Conclusions
0–10% discounts show positive average profits, suggesting they are safe and potentially effective for stimulating sales.

20–30% and above show significant erosion of profitability, with the >40% bracket being the most damaging.

The relationship is not linear but sharply negative after a tipping point at 10–20%.

 Strategic Recommendations
- Minimize usage of high discounts (>30%) as they consistently reduce profitability.
- Promote low-tier discounts (0–10%) where margins are maintained and profitability remains positive.
- Investigate high-discount, high-sales products for margin leakage.


### ---- 📊Monthly Financial Performance: Sales vs. Profit Over Time ----

In [ ]:
# Extract month from order date
df['Order Month'] = df['Order Date'].dt.to_period('M').astype(str)

# Aggregate monthly sales and profit
monthly = df.groupby(['Order Month']).agg({
    'Sales': 'sum',
    'Profit': 'sum'
}).reset_index()

# Plot sales and profit trends over time
plt.figure(figsize=(14, 6))
sns.lineplot(data=monthly, x='Order Month', y='Sales', label='Sales')
sns.lineplot(data=monthly, x='Order Month', y='Profit', label='Profit')
plt.xticks(rotation=45)
plt.title('Sales and Profit Performance Over Time')
plt.ylabel('Value')
plt.xlabel('Month')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

### ---- 📊Standard Deviation of Shipping Delay by State ----

In [ ]:
def analyze_profit_by_variable(data, group_by_column, kind_char, color_name):
    """
    General-purpose function to visualize total profit by a specific variable.

    Parameters:
        data: DataFrame containing the data.
        group_by_column: Column name to group by (e.g., 'Category', 'Region').
        kimd_char: Type of chart required (e.g., bar, barh, pie).
        color_name: A color name or list for consistent plot styling.

    Returns:
        grouped_data: A DataFrame with grouped and sorted profit values.
    """

    # Grouping and aggregating profit
    grouped_data = data.groupby(group_by_column)['Profit'].sum().reset_index()
    grouped_data = grouped_data.sort_values('Profit', ascending=(len(grouped_data) <= 5))

    # Plotting
    plt.figure(figsize=(10, 6) if len(grouped_data) <= 10 else (12, 7))
    grouped_data.plot(kind=kind_char, x=group_by_column, y='Profit', color=color_name , legend=False)
    plt.axvline(0, color='red', linestyle='-', linewidth=0.5)
    plt.xlabel('Total Profit')
    plt.ylabel(group_by_column)


    plt.title(f'Total Profit by {group_by_column}')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

    return grouped_data

### ---- 📊Profit Analysis of Furniture in the Western Region by Sub-Category ----

In [ ]:
furniture_west = df[(df['Category']=='Furniture') & (df['Region']=='West')]
analyze_profit_by_variable(furniture_west, 'Sub-Category', 'bar', ['Purple','pink'])

### ---- 📊Total Profit by Main Category ----

In [ ]:
analyze_profit_by_variable(df, 'Category', 'barh', ['skyblue', 'salmon', 'lightgreen'])

#### 🕵️‍♂️ Conclusions
This horizontal bar chart reveals that some categories might be yielding negative or low profits, potentially due to cost structure or discounting.

### ---- 📊Total Profit by Sub-Category ----

In [ ]:
analyze_profit_by_variable(df, 'Sub-Category', 'barh','teal')

It highlights which specific product types drive or drain profit.
Helps in optimizing product mix and identifying loss-making items.

### ---- 📊 Profit Share by Region ----

In [ ]:
region_profit = df.groupby('Region')['Profit'].sum().sort_values()
region_profit.plot(kind='pie', autopct='%1.1f%%', figsize=(8,8), colors=['brown','pink'], shadow=True)
plt.title('Profit Share by Region')
plt.show()

it informs regional performance in total profits.

### ---- 📊 Total Profit by Customer Segment ----

In [ ]:
segment_profit = df.groupby('Segment')['Profit'].sum()
segment_profit.plot(kind='bar', color=['gold', 'violet', 'lightblue'])
plt.title('Total Profit by Customer Segment')
plt.ylabel('Total Profit')
plt.xlabel('Customer Segment')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()


Customer segments differ in profitability. If a segment like “Home Office” consistently shows lower profit, strategies might include revising targeted offers or reconsidering discounts.

# 🔍 *Advanced Multivariate Analysis*

### ---- Shipping Delay by Region and Shipping Mode ----

### ---- 📊 Impact of Discount on Product Sales by Segment ----

In [ ]:
discount_product_segment = df.groupby(['Segment', 'Product Name']) \
                             .agg({'Sales': 'sum', 'Total Discount': 'sum'}) \
                             .reset_index()

plt.figure(figsize=(15, 8))
sns.scatterplot(data=discount_product_segment,
                x='Total Discount',
                y='Sales',
                hue='Segment',
                alpha=0.7)
plt.title('Impact of Discount on Product Sales by Segment')
plt.xlabel('Total Discount per Product')
plt.ylabel('Total Sales per Product')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions
This scatter plot reveals how discounting strategies correlate with total product sales across different customer segments:

Positive Cluster: Some products exhibit a positive relationship—higher discounts lead to higher total sales. This is more evident in segments like "Consumer," where promotions likely drive volume.

Saturation Zone: Beyond a certain discount threshold, sales gains flatten. This suggests diminishing returns and highlights the importance of controlled discounting.

Segment Differences: Each segment behaves differently—"Corporate" and "Home Office" segments may not respond to discounts as aggressively as "Consumer."

#### 🕵️ Recommendation :

Watch closely the ‘Consumer’ segment: discounts seem to lure them in like moths to a flame. But tread carefully—discounts beyond 30% may no longer yield fruit. For ‘Corporate’ and ‘Home Office’, investigate other sales levers—perhaps service or product bundling.

### ---- 📊Relationship Between Sales and Profit by Category with Quantity Impact ----

In [ ]:
plt.figure(figsize=(12, 7))

scatter = sns.scatterplot(
    data=df,
    x='Sales',
    y='Profit',
    hue='Category', 
    size='Quantity',  
    alpha=0.7,
    sizes=(40, 400),
    edgecolor='black',
    linewidth=0.5
)

plt.title('Relationship Between Sales and Profit by Category with Quantity Impact')
plt.xlabel('Sales')
plt.ylabel('Profit')
plt.axhline(0, color='gray', linestyle='--')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions 
Sales vs. Profit Behavior:

The scatter shows a generally positive correlation between sales and profit—however, this is not linear, and many high-sale entries still suffer from low or negative profits.

The horizontal reference line at 0 helps reveal products that are sold at a loss.

Role of Quantity:

Larger dots (representing higher quantities) do not necessarily correspond to higher profits, indicating that some products may be over-discounted or mispriced.

Category-Based Patterns:

Categories are clearly separated:

One may notice that [e.g., Office Supplies] has many entries near zero profit despite decent sales—potential over-discounting or margin issues.

Another, like [Technology], may show fewer entries but higher profitability, hinting at higher margins.

Outliers:

Some items show extremely high sales but low or even negative profit. These warrant investigation to determine if they are being used as loss leaders or if pricing errors exist.
Conclusion
This multi-dimensional visualization reveals that:

High sales volumes are not always profitable.

Quantity sold alone cannot explain profitability—Category and perhaps discounting policy play crucial roles.

### ---- 📊The relationship between (product + segment + region) and sales ----

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(data=df, x='Segment', y='Sales', hue='Region')
plt.yscale('log')
plt.title('Sales by Segment and Region')
plt.ylabel('Sales (logarithmic scale)')
plt.xlabel('Segment')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#### 🕵️‍♂️ Conclusions
Segment Insights:

Consumer and Corporate segments tend to show higher median sales compared to Home Office, particularly in the West and East regions.

The Home Office segment displays greater variability and more lower-end sales.

Regional Variations:

The West and East regions generally show higher and more stable sales distributions across segments.

The South and Central regions exhibit more variability and frequent low-value orders.

Outliers:

Many segments contain high-value outliers, likely large business-to-business orders.

#### 🧠 Conclusion
This plot highlights disparities in sales performance across customer segments and regions.

The Consumer and Corporate segments in East and West regions are top performers.

The Home Office segment may benefit from more targeted marketing or bundling to improve order value.

### ---- Relationship Between Discount and Profit ----

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Discount', y='Profit')
plt.title('Relationship Between Discount and Profit')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🧠 Insight:
This scatterplot reveals the general impact of discounting on profit. As a detective of margins, you’re likely to find that excessive discounting (especially near 0.8–1.0) is correlated with negative profits — an early red flag for strategic review.

### ---- Relationship Between Quantity and Profit ----

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Quantity', y='Profit')
plt.title('Relationship Between Quantity and Profit')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🧠 Insight:
An essential check for economies of scale. If increasing quantity doesn’t translate to profit, the cost structure or pricing model may be flawed.

### ---- Relationship Between Sales and Profit ----

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Sales', y='Profit')
plt.title('Relationship Between Sales and Profit')
plt.xscale('log')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🧠 Insight:
You may discover that even high sales can sometimes result in loss — this suggests pricing misalignment or high associated costs.

### ---- Loss-Making Products and States ----

In [ ]:
loss_products = df[df['Profit'] < 0]

plt.figure(figsize=(12, 6))
sns.countplot(data=loss_products, y='Sub-Category', order=loss_products['Sub-Category'].value_counts().index)
plt.title('Loss-Making Sub-Categories')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(data=loss_products, y='State', order=loss_products['State'].value_counts().nlargest(15).index)
plt.title('Top Loss-Making States')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🧠 Insight:
This narrows down your suspects: which products and which states bleed profits. These are the crime scenes of your sales strategy — targets for discontinuation, reevaluation, or localized corrective pricing.

### ---- Strategic Profit Insights ----

In [ ]:
high_profit = df.groupby(['Category', 'Region', 'Segment'])['Profit'].sum().reset_index().sort_values(by='Profit', ascending=False)
print("\nTop Profit-Generating Combinations:")
print(high_profit.head(10))

low_profit = df.groupby(['Sub-Category', 'State'])['Profit'].sum().reset_index().sort_values(by='Profit')
print("\nLowest Profit or Loss-Making Combinations:")
print(low_profit.head(10))

#### 🧠 Insight:
This gives a top-down view of what’s working well — your most profitable customer-region-product intersections. Vital for focusing marketing and resource allocation.

### ---- Shipping Efficiency Analysis ----

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df['Ship Duration'] = (df['Ship Date'] - df['Order Date']).dt.days

plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Ship Mode', y='Ship Duration')
plt.title('Shipping Duration by Ship Mode')
plt.ylabel('Shipping Days')
plt.xlabel('Shipping Mode')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

#### 🧠 Insight:
Boxplot reveals outliers and consistency. If standard shipping regularly takes longer than promised, that’s a service issue. Faster shipping with shorter durations but low usage might need promotion.